In [1]:
# load env variables
from datetime import datetime
import os
from dotenv import load_dotenv
load_dotenv()
# set local dir logs based on date _${now:%Y-%m-%d}_${now:%H-%M-%S}
os.environ["SLURM_JOB_ID"] = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

In [2]:
import hydra
import torch
from src.commons.constants import PROJECT_PATH
from omegaconf import DictConfig, OmegaConf

Welcome in JZAY


In [4]:
def load_config():
    # Initialize the Hydra configuration
    hydra.initialize(config_path="../configs", version_base=None)
    
    # Compose the configuration with the desired environment override
    cfg = hydra.compose(
        config_name="train", 
        overrides=["experiment=adapter_concat", 
                   "sam_type=small", 
                   "data=levir-cd",
                   "data.params.batch_size=1"
                  ])
    
    return cfg

def get_dloader(mode: str, dmodule):

    def wrap_mode(mode):
        if mode == "train":
            return "fit"
        return mode
    if not dmodule.ds_dict_type:
        mode_ = wrap_mode(mode)
        dmodule.setup(mode_)
    factory_dl = {
        "train": dmodule.train_dataloader,
        "val": dmodule.val_dataloader,
        "test": dmodule.test_dataloader,
    }
    return factory_dl[mode]()

In [5]:
from hydra.core.global_hydra import GlobalHydra
GlobalHydra.instance().clear()
cfg = load_config()

In [6]:
module = hydra.utils.instantiate(cfg.model.instance)

INIT ADAPTER VIT


2024-08-13 20:45:18,587 - INFO ::  Weights loaded for : ['image_encoder']


In [7]:
data_module = hydra.utils.instantiate(cfg.data)

In [8]:
dloader = get_dloader("test", data_module)

In [9]:
batch = next(iter(dloader))

In [11]:
preds, _ = module.model(batch, multimask_output=False)
preds = preds.squeeze()

In [12]:
preds.shape

torch.Size([1024, 1024])

In [15]:
img_embeddings = module.model.image_embeddings